In [3]:
import torch
import torchvision

print("Versão do PyTorch:", torch.__version__)
print("Versão do Torchvision:", torchvision.__version__)
print("CUDA disponível:", torch.cuda.is_available())
print("Placa de vídeo:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Nenhuma")

Versão do PyTorch: 2.5.1+cu124
Versão do Torchvision: 0.20.1+cu124
CUDA disponível: True
Placa de vídeo: NVIDIA GeForce GTX 1660 SUPER


In [4]:
import sys
import os

# Como estamos em 'notebooks', precisamos subir um nível para acessar 'src'
projeto_path = os.path.dirname(os.getcwd())  # Sobe um nível
sys.path.append(projeto_path)

### EasyOCR model

In [ ]:
import cv2
from ultralytics import YOLO
from time import time
from src import OCRdetect  # EasyOCR
import csv

def process_video(model_path, video_path, confiance=0.5):
    model = YOLO(model_path)
    cap = cv2.VideoCapture(video_path)

    with open('analitycs/easyocr_frame_metrics.csv', 'w', newline='') as frame_csv, \
         open('analitycs/easyocr_detection_metrics.csv', 'w', newline='') as detect_csv:

        frame_writer = csv.DictWriter(frame_csv, fieldnames=[
            'frame_number', 'yolo_time', 'total_processing_time', 'num_detections'
        ])
        frame_writer.writeheader()

        detect_writer = csv.DictWriter(detect_csv, fieldnames=[
            'frame_number', 'detection_index', 'ocr_time', 
            'plate_detected', 'plate_text', 'confidence', 'class'
        ])
        detect_writer.writeheader()

        frame_counter = 0

        while cap.isOpened():
            frame_counter += 1
            frame_start_time = time()

            ret, frame = cap.read()
            if not ret: break

            # YOLO
            yolo_start = time()
            results = model.predict(frame, conf=confiance, device="gpu")
            yolo_time = time() - yolo_start

            detections = results[0].boxes.xyxy.cpu().tolist()
            confs = results[0].boxes.conf.cpu().tolist()
            classes = results[0].boxes.cls.cpu().tolist()

            valid_detections = [
                (detections[i], confs[i], classes[i]) 
                for i in range(len(detections)) if confs[i] > confiance
            ]

            for detection_idx, (box, conf, cls) in enumerate(valid_detections):
                x1, y1, x2, y2 = box
                crop_obj = frame[int(y1):int(y2), int(x1):int(x2)]
                
                try:
                    h, w = crop_obj.shape[:2]
                    crop_obj = cv2.resize(crop_obj, (w*4, h*4), interpolation=cv2.INTER_LANCZOS4)
                except: continue

                # EasyOCR
                ocr_start = time()
                plate = OCRdetect.detect_text(crop_obj)
                ocr_time = time() - ocr_start

                plate_detected = len(plate) > 0
                plate_text = '|'.join(plate) if plate_detected else ''

                detect_writer.writerow({
                    'frame_number': frame_counter,
                    'detection_index': detection_idx,
                    'ocr_time': ocr_time,
                    'plate_detected': plate_detected,
                    'plate_text': plate_text,
                    'confidence': conf,
                    'class': cls
                })

            frame_writer.writerow({
                'frame_number': frame_counter,
                'yolo_time': yolo_time,
                'total_processing_time': time() - frame_start_time,
                'num_detections': len(valid_detections)
            })

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    model_path = "C:/projetosML/auto_Plate_Detection/outputs/modelos_treinados/modelo_br_dataArgumetetion_40epochs.pt"
    video_path = r"C:\projetosML\auto_Plate_Detection\inputs\simulation.mkv"
    process_video(model_path, video_path)

### TesseractOCR

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from time import time
from src import tesseractOCR  #TesseractOCR
import csv

def process_video(model_path, video_path, confiance=0.5):
    model = YOLO(model_path)
    cap = cv2.VideoCapture(video_path)

    with open('analitycs/tesseract_frame_metrics.csv', 'w', newline='') as frame_csv, \
         open('analitycs/tesseract_detection_metrics.csv', 'w', newline='') as detect_csv:

        frame_writer = csv.DictWriter(frame_csv, fieldnames=[
            'frame_number', 'yolo_time', 'total_processing_time', 'num_detections'
        ])
        frame_writer.writeheader()

        detect_writer = csv.DictWriter(detect_csv, fieldnames=[
            'frame_number', 'detection_index', 'ocr_time', 
            'plate_detected', 'plate_text', 'confidence', 'class'
        ])
        detect_writer.writeheader()

        frame_counter = 0

        while cap.isOpened():
            frame_counter += 1
            frame_start_time = time()

            ret, frame = cap.read()
            if not ret: break

            # YOLO
            yolo_start = time()
            results = model.predict(frame, conf=confiance, device="gpu")
            yolo_time = time() - yolo_start

            detections = results[0].boxes.xyxy.cpu().tolist()
            confs = results[0].boxes.conf.cpu().tolist()
            classes = results[0].boxes.cls.cpu().tolist()

            valid_detections = [
                (detections[i], confs[i], classes[i]) 
                for i in range(len(detections)) if confs[i] > confiance
            ]

            for detection_idx, (box, conf, cls) in enumerate(valid_detections):
                x1, y1, x2, y2 = box
                crop_obj = frame[int(y1):int(y2), int(x1):int(x2)]
                
                try:
                    h, w = crop_obj.shape[:2]
                    crop_obj = cv2.resize(crop_obj, (w*4, h*4), interpolation=cv2.INTER_LANCZOS4)
                except: continue

                # TesseractOCR
                ocr_start = time()
                plate = tesseractOCR.detect_text(crop_obj)
                ocr_time = time() - ocr_start

                plate_detected = len(plate) > 0 and plate[0] != ''
                plate_text = plate[0] if plate_detected else ''  # Pegar primeiro elemento

                detect_writer.writerow({
                    'frame_number': frame_counter,
                    'detection_index': detection_idx,
                    'ocr_time': ocr_time,
                    'plate_detected': plate_detected,
                    'plate_text': plate_text,
                    'confidence': conf,
                    'class': cls
                })

            frame_writer.writerow({
                'frame_number': frame_counter,
                'yolo_time': yolo_time,
                'total_processing_time': time() - frame_start_time,
                'num_detections': len(valid_detections)
            })

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    model_path = "C:/projetosML/auto_Plate_Detection/outputs/modelos_treinados/modelo_br_dataArgumetetion_40epochs.pt"
    video_path = r"C:\projetosML\auto_Plate_Detection\inputs\202501291141.mp4"
    process_video(model_path, video_path)

### PadleOCR

In [ ]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from time import time
from src import paddleOCRdetect  # PaddleOCR
import csv

def process_video(model_path, video_path, confiance=0.5):
    model = YOLO(model_path)
    cap = cv2.VideoCapture(video_path)

    with open('analitycs/paddleocr_frame_metrics.csv', 'w', newline='') as frame_csv, \
         open('analitycs/paddleocr_detection_metrics.csv', 'w', newline='') as detect_csv:

        frame_writer = csv.DictWriter(frame_csv, fieldnames=[
            'frame_number', 'yolo_time', 'total_processing_time', 'num_detections'
        ])
        frame_writer.writeheader()

        detect_writer = csv.DictWriter(detect_csv, fieldnames=[
            'frame_number', 'detection_index', 'ocr_time', 
            'plate_detected', 'plate_text', 'confidence', 'class'
        ])
        detect_writer.writeheader()

        frame_counter = 0

        while cap.isOpened():
            frame_counter += 1
            frame_start_time = time()

            ret, frame = cap.read()
            if not ret: break

            # YOLO
            yolo_start = time()
            results = model.predict(frame, conf=confiance, device="gpu")
            yolo_time = time() - yolo_start

            detections = results[0].boxes.xyxy.cpu().tolist()
            confs = results[0].boxes.conf.cpu().tolist()
            classes = results[0].boxes.cls.cpu().tolist()

            valid_detections = [
                (detections[i], confs[i], classes[i]) 
                for i in range(len(detections)) if confs[i] > confiance
            ]

            for detection_idx, (box, conf, cls) in enumerate(valid_detections):
                x1, y1, x2, y2 = box
                crop_obj = frame[int(y1):int(y2), int(x1):int(x2)]
                
                try:
                    h, w = crop_obj.shape[:2]
                    crop_obj = cv2.resize(crop_obj, (w*4, h*4), interpolation=cv2.INTER_LANCZOS4)
                except: continue

                # PaddleOCR
                ocr_start = time()
                plate = paddleOCRdetect.detect_text(crop_obj)
                ocr_time = time() - ocr_start

                plate_detected = len(plate) > 0
                plate_text = '|'.join(plate) if plate_detected else ''

                detect_writer.writerow({
                    'frame_number': frame_counter,
                    'detection_index': detection_idx,
                    'ocr_time': ocr_time,
                    'plate_detected': plate_detected,
                    'plate_text': plate_text,
                    'confidence': conf,
                    'class': cls
                })

            frame_writer.writerow({
                'frame_number': frame_counter,
                'yolo_time': yolo_time,
                'total_processing_time': time() - frame_start_time,
                'num_detections': len(valid_detections)
            })

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    model_path = "C:/projetosML/auto_Plate_Detection/outputs/modelos_treinados/modelo_br_dataArgumetetion_40epochs.pt"
    video_path = r"C:\projetosML\auto_Plate_Detection\inputs\202501291141.mp4"
    process_video(model_path, video_path)

### Analize dos resultados

In [6]:
import pandas as pd
import re

def validate_plate(text):
    # Limpa o texto, mantendo apenas letras e números, e converte para maiúsculas
    text = re.sub(r'[^A-Z0-9]', '', text.upper())
    
    # Verifica se o texto tem exatamente 6 caracteres após a limpeza
    if len(text) != 6:
        return False
    
    # Processa a parte das letras (posições 1 e 2) substituindo '0' por 'O'
    letters_part = text[1:3].replace('0', 'O')
    # Reconstroi o texto com as letras ajustadas
    processed_text = text[0] + letters_part + text[3:]
    
    # Verifica se o texto processado corresponde ao padrão da placa
    padrao_custom = re.match(r'^\d[A-Z]{2}\d{3}$', processed_text)
    return bool(padrao_custom)

def analyze_results(ocr_type):
    frame_metrics = pd.read_csv(f'analitycs/{ocr_type}_frame_metrics.csv')
    detect_metrics = pd.read_csv(f'analitycs/{ocr_type}_detection_metrics.csv')
    
    detect_metrics['valid_plate'] = detect_metrics['plate_text'].apply(
        lambda x: validate_plate(str(x))
    )
    
    total_valid = detect_metrics['valid_plate'].sum()
    avg_ocr_time = detect_metrics['ocr_time'].mean()
    avg_yolo_time = frame_metrics['yolo_time'].mean()
    
    return {
        'OCR Model': ocr_type,
        'Total Valid Plates': total_valid,
        'Accuracy (%)': (total_valid / len(detect_metrics)) * 100 if len(detect_metrics) > 0 else 0,
        'Avg OCR Time (ms)': avg_ocr_time * 1000,
        'Avg YOLO Time (ms)': avg_yolo_time * 1000,
        'Total Detections': len(detect_metrics)
    }

if __name__ == '__main__':
    easyocr_stats = analyze_results('easyocr')
    tesseract_stats = analyze_results('tesseract')
    paddleocr_stats = analyze_results('paddleocr')
    
    df_comparison = pd.DataFrame([easyocr_stats, tesseract_stats, paddleocr_stats])
    print("\n=== Comparação de Desempenho ===")
    print(df_comparison.to_markdown(index=False))
    
    # Salvar resultados em CSV
    df_comparison.to_csv('ocr_comparison.csv', index=False)


=== Comparação de Desempenho ===
| OCR Model   |   Total Valid Plates |   Accuracy (%) |   Avg OCR Time (ms) |   Avg YOLO Time (ms) |   Total Detections |
|:------------|---------------------:|---------------:|--------------------:|---------------------:|-------------------:|
| easyocr     |                 3521 |        17.0906 |             58.771  |              30.7459 |              20602 |
| tesseract   |                 3302 |        16.1475 |            263.429  |              56.7266 |              20449 |
| paddleocr   |                11560 |        56.5336 |             49.1595 |              36.4519 |              20448 |


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.ticker import MaxNLocator

# Verificar estilos disponíveis e configurar
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("pastel")
MODELOS = ['easyocr', 'tesseract', 'paddleocr']

def criar_graficos_avancados():
    # Criar pasta para os gráficos
    os.makedirs('analise_visual', exist_ok=True)
    
    # Carregar e combinar dados de todos os modelos
    dados_completos = []
    for modelo in MODELOS:
        # Ler dados de detecção
        deteccao = pd.read_csv(f'analitycs/{modelo}_detection_metrics.csv')
        deteccao['modelo'] = modelo
        
        # Ler dados de frames
        frames = pd.read_csv(f'analitycs/{modelo}_frame_metrics.csv')
        frames['modelo'] = modelo
        
        dados_completos.append(deteccao)
        dados_completos.append(frames)
    
    df_full = pd.concat(dados_completos, ignore_index=True)
    
    # 1. Gráfico de Desempenho Temporal por Modelo
    plt.figure(figsize=(15, 8))
    for modelo in MODELOS:
        # Dados de frames
        frame_data = df_full[(df_full['modelo'] == modelo) & (df_full['frame_number'].notnull())]
        plt.plot(frame_data['frame_number'], frame_data['yolo_time'].rolling(10).mean(), 
                 label=f'{modelo} - YOLO', linestyle='--')
        plt.plot(frame_data['frame_number'], frame_data['total_processing_time'].rolling(10).mean(),
                 label=f'{modelo} - Total', linewidth=2)
    
    plt.title('Desempenho Temporal dos Modelos (Média Móvel de 10 frames)')
    plt.xlabel('Número do Frame')
    plt.ylabel('Tempo de Processamento (s)')
    plt.legend()
    plt.savefig('analise_visual/desempenho_temporal.png', bbox_inches='tight')
    plt.close()
    
    # 2. Distribuição de Confiança nas Detecções
    plt.figure(figsize=(12, 6))
    for modelo in MODELOS:
        dados_modelo = df_full[(df_full['modelo'] == modelo) & (df_full['confidence'].notnull())]
        sns.kdeplot(dados_modelo['confidence'], label=modelo, fill=True, alpha=0.3)
    
    plt.title('Distribuição de Confiança nas Detecções por Modelo')
    plt.xlabel('Confiança da Detecção')
    plt.ylabel('Densidade')
    plt.legend()
    plt.savefig('analise_visual/distribuicao_confianca.png')
    plt.close()
    
    # 3. Comparação de Detecções Válidas vs Inválidas
    plt.figure(figsize=(10, 6))
    validas = []
    for modelo in MODELOS:
        deteccao = pd.read_csv(f'analitycs/{modelo}_detection_metrics.csv')
        validas.append(deteccao['plate_detected'].sum())
    
    plt.bar(MODELOS, validas, label='Válidas')
    plt.bar(MODELOS, [len(pd.read_csv(f'analitycs/{m}_detection_metrics.csv')) - v for m,v in zip(MODELOS, validas)], 
            bottom=validas, label='Inválidas')
    plt.title('Detecções Válidas vs Inválidas por Modelo')
    plt.xlabel('Modelo OCR')
    plt.ylabel('Quantidade de Detecções')
    plt.legend()
    plt.savefig('analise_visual/validas_vs_invalidas.png')
    plt.close()
    
    # 4. Análise de Desempenho por Frame
    fig, axs = plt.subplots(3, 1, figsize=(15, 12))
    for idx, modelo in enumerate(MODELOS):
        frame_data = pd.read_csv(f'analitycs/{modelo}_frame_metrics.csv')
        ax = axs[idx]
        
        # Gráfico de linhas para tempos de processamento
        ax.plot(frame_data['frame_number'], frame_data['yolo_time'], 
                label='Tempo YOLO', alpha=0.7)
        ax.plot(frame_data['frame_number'], frame_data['total_processing_time'], 
                label='Tempo Total', linewidth=2)
        
        # Gráfico de barras para número de detecções
        ax2 = ax.twinx()
        ax2.bar(frame_data['frame_number'], frame_data['num_detections'], 
                alpha=0.3, color='red', label='Detecções')
        
        ax.set_title(f'Desempenho do {modelo.capitalize()} por Frame')
        ax.set_xlabel('Frame')
        ax.set_ylabel('Tempo (s)')
        ax2.set_ylabel('Número de Detecções')
        ax.legend(loc='upper left')
        ax2.legend(loc='upper right')
    
    plt.tight_layout()
    plt.savefig('analise_visual/desempenho_por_frame.png')
    plt.close()
    
    # 5. Heatmap de Detecções por Frame
    plt.figure(figsize=(15, 6))
    heatmap_data = pd.pivot_table(df_full[df_full['frame_number'].notnull()], 
                                values='num_detections', 
                                index='frame_number', 
                                columns='modelo')
    sns.heatmap(heatmap_data.T, cmap='viridis', cbar_kws={'label': 'Número de Detecções'})
    plt.title('Frequência de Detecções por Frame e Modelo')
    plt.xlabel('Número do Frame')
    plt.ylabel('Modelo OCR')
    plt.savefig('analise_visual/heatmap_deteccoes.png')
    plt.close()
    
    # 6. Comparação de Tempos de Processamento
    plt.figure(figsize=(12, 6))
    tempos = []
    for modelo in MODELOS:
        dados = pd.read_csv(f'analitycs/{modelo}_frame_metrics.csv')
        tempos.append({
            'modelo': modelo,
            'YOLO': dados['yolo_time'].mean(),
            'OCR': dados['total_processing_time'].mean() - dados['yolo_time'].mean()
        })
    
    df_tempos = pd.DataFrame(tempos).set_index('modelo')
    df_tempos.plot(kind='bar', stacked=True, ax=plt.gca())
    plt.title('Distribuição de Tempos de Processamento por Modelo')
    plt.ylabel('Tempo Médio (s)')
    plt.xticks(rotation=0)
    plt.savefig('analise_visual/tempos_processamento.png')
    plt.close()

if __name__ == '__main__':
    criar_graficos_avancados()
    print("Análise visual completa! Verifique a pasta 'analise_visual'")

Análise visual completa! Verifique a pasta 'analise_visual'


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("pastel")
MODELOS = ['easyocr', 'tesseract', 'paddleocr']
CORES = {'easyocr': '#66c2a5', 'tesseract': '#fc8d62', 'paddleocr': '#8da0cb'}

def validate_plate(text):
    text = re.sub(r'[^A-Z0-9]', '', text.upper())
    if len(text) != 6:
        return False
    letters_part = text[1:3].replace('0', 'O')
    processed_text = text[0] + letters_part + text[3:]
    return bool(re.match(r'^\d[A-Z]{2}\d{3}$', processed_text))

def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    return df[(df[column] <= (Q3 + 1.5 * IQR))]

def criar_graficos_final():
    os.makedirs('analise_final', exist_ok=True)
    
    # Dados consolidados
    dados_gerais = []
    dados_validacoes = []
    
    for modelo in MODELOS:
        # Carregar e limpar dados
        deteccao = pd.read_csv(f'analitycs/{modelo}_detection_metrics.csv')
        frame = pd.read_csv(f'analitycs/{modelo}_frame_metrics.csv')
        frame = remove_outliers(frame, 'total_processing_time')
        
        # Processar validações
        deteccao['valid_format'] = deteccao['plate_text'].apply(lambda x: validate_plate(str(x)))
        validas_ocr = deteccao['plate_detected'].sum()
        validas_formato = deteccao['valid_format'].sum()
        
        # Salvar dados
        dados_gerais.append({
            'Modelo': modelo.upper(),
            'OCR Validas': validas_ocr,
            'Formato Valido': validas_formato,
            'Total Detecções': len(deteccao)
        })
        
        # Preparar dados para gráficos de validação
        dados_validacoes.append({
            'Modelo': modelo.upper(),
            'Tipo': 'Validação OCR',
            'Quantidade': validas_ocr
        })
        dados_validacoes.append({
            'Modelo': modelo.upper(),
            'Tipo': 'Formato Correto',
            'Quantidade': validas_formato
        })
    
    df_geral = pd.DataFrame(dados_gerais)
    df_validacoes = pd.DataFrame(dados_validacoes)
    
    # 1. Comparação de Validações
    plt.figure(figsize=(12,6))
    sns.barplot(x='Modelo', y='Quantidade', hue='Tipo', data=df_validacoes)
    plt.title('Comparação de Detecções Validadas vs Formato Correto')
    plt.ylabel('Quantidade')
    plt.legend(title='Tipo de Validação')
    plt.savefig('analise_final/comparacao_validacoes.png', bbox_inches='tight')
    plt.close()
    
    # 2. Evolução Temporal Limpa
    plt.figure(figsize=(12,6))
    for modelo in MODELOS:
        frame_data = pd.read_csv(f'analitycs/{modelo}_frame_metrics.csv')
        frame_data = remove_outliers(frame_data, 'total_processing_time')
        
        sns.lineplot(
            x='frame_number', 
            y='total_processing_time', 
            data=frame_data,
            label=modelo.upper(),
            color=CORES[modelo],
            estimator='median',
            errorbar=None,
            linewidth=1.5
        )
    
    plt.title('Evolução do Tempo de Processamento (Outliers Removidos)')
    plt.xlabel('Número do Frame')
    plt.ylabel('Tempo Total (s)')
    plt.legend()
    plt.savefig('analise_final/evolucao_temporal_limpa.png', bbox_inches='tight')
    plt.close()
    
    # 3. Gráfico de Confiança Simplificado
    plt.figure(figsize=(12,6))
    for modelo in MODELOS:
        dados = pd.read_csv(f'analitycs/{modelo}_detection_metrics.csv')
        dados['Valido'] = dados['plate_detected'].apply(lambda x: 'Válida' if x else 'Inválida')
        
        sns.boxplot(
            x='Modelo', 
            y='confidence', 
            hue='Valido',
            data=dados.assign(Modelo=modelo.upper()),
            palette={'Válida': CORES[modelo], 'Inválida': '#ff0000'},
            width=0.4
        )
    
    plt.title('Distribuição de Confiança por Validade (OCR)')
    plt.xlabel('Modelo')
    plt.ylabel('Confiança')
    plt.legend(title='Validação')
    plt.savefig('analise_final/boxplot_confianca.png', bbox_inches='tight')
    plt.close()
    
    # 4. Taxa de Acerto no Formato
    plt.figure(figsize=(10,6))
    df_geral['Taxa Acerto'] = (df_geral['Formato Valido'] / df_geral['Total Detecções']) * 100
    sns.barplot(x='Modelo', y='Taxa Acerto', data=df_geral, palette=CORES.values())
    
    for index, row in df_geral.iterrows():
        plt.text(index, row['Taxa Acerto'] + 1, f"{row['Taxa Acerto']:.1f}%", ha='center')
    
    plt.title('Taxa de Acerto no Formato da Placa')
    plt.ylabel('Taxa de Acerto (%)')
    plt.ylim(0, 100)
    plt.savefig('analise_final/taxa_acerto_formato.png', bbox_inches='tight')
    plt.close()

if __name__ == '__main__':
    criar_graficos_final()
    print("Análise final gerada na pasta 'analise_final'!")

C:\Users\danie\AppData\Local\Temp\ipykernel_17236\2328361898.py:125: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Modelo', y='Taxa Acerto', data=df_geral, palette=CORES.values())


Análise final gerada na pasta 'analise_final'!


In [10]:
def criar_grafico_bottleneck():
    os.makedirs('analise_final', exist_ok=True)
    
    # Dados de processamento
    dados_processamento = []
    
    for modelo in MODELOS:
        # Carregar dados limpos
        frame_data = pd.read_csv(f'analitycs/{modelo}_frame_metrics.csv')
        frame_data = remove_outliers(frame_data, 'yolo_time')
        deteccao_data = pd.read_csv(f'analitycs/{modelo}_detection_metrics.csv')
        
        # Calcular métricas-chave
        media_yolo = frame_data['yolo_time'].mean() * 1000  # ms
        media_ocr = deteccao_data['ocr_time'].mean() * 1000  # ms por detecção
        deteccoes_por_frame = frame_data['num_detections'].mean()
        
        # Calcular contribuição total
        contrib_yolo = media_yolo
        contrib_ocr_total = media_ocr * deteccoes_por_frame
        
        dados_processamento.extend([
            {'Modelo': modelo.upper(), 'Componente': 'YOLO', 'Tempo (ms)': contrib_yolo},
            {'Modelo': modelo.upper(), 'Componente': 'OCR', 'Tempo (ms)': contrib_ocr_total}
        ])
    
    df_processamento = pd.DataFrame(dados_processamento)
    
    # Gráfico de Comparação
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x='Modelo', y='Tempo (ms)', hue='Componente', data=df_processamento, palette='coolwarm')
    
    # Anotações personalizadas
    for p in ax.patches:
        ax.annotate(f"{p.get_height():.1f} ms", 
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha='center', va='center', 
                   xytext=(0, 7), 
                   textcoords='offset points',
                   fontsize=9)
    
    plt.title('Contribuição do YOLO vs OCR no Tempo Total por Frame')
    plt.ylabel('Tempo Médio por Frame (ms)')
    plt.legend(title='Componente')
    plt.grid(axis='y', alpha=0.3)
    plt.savefig('analise_final/bottleneck_analysis.png', bbox_inches='tight')
    plt.close()

# Adicionar esta chamada no final da função criar_graficos_final()
criar_grafico_bottleneck()